This script has the following objectives:  
- Joining the OUI tables together (derived.oui and meta.14112022_oui) to get a maximum number of device captured  
- Extracting the data and cleaning it, only keeping the mobile devices that are associated

The principal aim is to filter out the associated devices and use that as a baseline to add on afterward...

In [5]:
#saving bigtable
import pandas_gbq
from datalab.context import Context
import datalab.storage as storage
import datalab.bigquery as bq
from pandas import DataFrame
import time
from tqdm import tqdm
tqdm.pandas()

In [11]:
%%bigquery df_onroute_visit_raw
with oui_merged as(
Select COALESCE(oui.oui, oui_parsed.oui) as oui, COALESCE(oui.companyName, oui_parsed.companyName) as companyName
from data-prod-270222.meta.oui oui
full outer join data-prod-270222.derived.14102022_oui oui_parsed on utils.normalize_mac(oui.oui) = utils.normalize_mac(oui_parsed.oui)
)
SELECT scout.device_mac, scout.ap_oem, scout.Timestamp_ms, scout.Timestamp, datetime(timestamp_seconds(scout.timestamp)) as local_time,
scout.signal_strength, scout.noise_floor, scout.channel, hard.brand_name, hard.market_segment, scout.ap_mac, hard.site_uuid, oui.*
FROM  data-prod-270222.raw.rtls_scout_raw scout
join meta.hardware_details as hard on utils.normalize_mac(scout.ap_mac) = utils.normalize_mac(hard.mac)
join oui_merged as oui on SUBSTRING(scout.device_mac, 1, 6) like utils.normalize_mac(oui.oui)
where scout.timestamp between 1639756800 and 1639760400
and hard.site_uuid = '771f6b8e-7cfb-44ff-bc68-d0233ebf8db8'
and companyName IN ('Apple, Inc', 'Apple, Inc.', 'Google, Inc', 'Huawei Tech Co, Ltd', 'Intel Corp', 'Tct mobile Ltd', 
                    'LG Electronics (Mobile Communications)', 'LG innotek', 'Samsung Electro-Mechanics(Thailand)',
                    'Motorola Mobility Llc, a Lenovo Co', 'Samsung Electronics Co, Ltd', 'vivo Mobile Communication Co, Ltd',
                    'Mobiwire Mobiles (Ningbo) Co, Ltd', 'Motorola (Wuhan) Mobility Tech Communication Co, Ltd',
                     'Samsung Electro-Mechanics(Thailand)')
order by scout.timestamp
limit 100000

Downloading: 100%|██████████| 7972/7972 [00:00<00:00, 12034.42rows/s]


In [ ]:
len(set(df_onroute_visit_raw['device_mac']))

{'ALPSALPINE CO .,LTD',
 'Alps Electric Co, Ltd',
 'Amazon Tech Inc',
 'Ampak Tech, Inc',
 'Apple, Inc',
 'Apple, Inc.',
 'Aruba, a Hewlett Packard Enterprise Co',
 'Chongqing Fugui Electronics Co, Ltd',
 'Cisco Meraki',
 'Cisco Systems, Inc',
 'Cisco-Linksys, Llc',
 'Continental Automotive Systems Inc',
 'CyberTAN Tech Inc',
 'D-Link International',
 'Danlaw Inc',
 'Epigram, Inc',
 'Espressif Inc',
 'Essys',
 'Ezurio Ltd',
 'FN-Link Tech Ltd',
 'Ford Motor Co',
 'Garmin International',
 'Google, Inc',
 'Guangdong Huiwei High-Tech Co, Ltd',
 'Guangdong Oppo Mobile Telecommunications Corp, Ltd',
 'HP Inc',
 'Harman/Becker Automotive Systems GmbH',
 'Hewlett Packard',
 'Hon Hai Precision Ind. Co, Ltd',
 'Huawei Tech Co, Ltd',
 'Ieee Registration Authority',
 'Intel Corp',
 'Isaac Instruments',
 'LG Electronics (Mobile Communications)',
 'LG innotek',
 'LM Tech Ltd',
 'LS Research, Llc',
 'MediaTek Inc',
 'Microsoft Corp',
 'Mitsumi Electric Co, Ltd',
 'Motorola (Wuhan) Mobility Tech Comm

In [8]:
start = time.time()
df_onroute_visit_raw.to_gbq('data-prod-270222.datascience.counting_baseline_raw', 
                 'data-prod-270222',
                 chunksize=10000, 
                 if_exists='replace'
                 )
end = time.time()
print("time alternative 1 " + str(end - start))

100%|██████████| 1/1 [00:00<00:00, 10618.49it/s]

time alternative 1 4.658047199249268
